In [22]:
# import libraries
import psycopg2
import pandas as pd

# get username and password for DB
from config import sql_u
from config import sql_pw
from config import sql_host


In [23]:
# connect to DB
conn = psycopg2.connect(
    host=sql_host,
    port='5432',
    database='postgres',
    user=sql_u,
    password=sql_pw)

cur = conn.cursor()

In [24]:
# Get styles table
# run query
sql_query = '''SELECT *
               FROM styles;
            '''

cur.execute(sql_query)

# convert query results to DF
styles_df = pd.DataFrame(cur.fetchall())

# add column names to DF
num_cols = len(cur.description)
col_names = [i[0] for i in cur.description]
styles_df.columns = col_names


In [25]:
# Get recipes table
# run query
sql_query = '''SELECT *
               FROM recipes;
            '''

cur.execute(sql_query)

# convert query results to DF
recipes_df = pd.DataFrame(cur.fetchall())

# add column names to DF
num_cols = len(cur.description)
col_names = [i[0] for i in cur.description]
recipes_df.columns = col_names

In [26]:
# Get reviews table
# run query
sql_query = '''SELECT *
               FROM reviews;
            '''

cur.execute(sql_query)

# convert query results to DF
reviews_df = pd.DataFrame(cur.fetchall())

# add column names to DF
num_cols = len(cur.description)
col_names = [i[0] for i in cur.description]
reviews_df.columns = col_names

In [27]:
# close sql connection
cur.close()
conn.close()

In [31]:
# Take a look at ALL columns of each dataframe
pd.set_option('display.max_columns', None)


In [32]:
# take a look at the styles_df
styles_df.head()

,style,styleid
0,Altbier,1
1,Alternative Grain Beer,2
2,Alternative Sugar Beer,3
3,American Amber Ale,4
4,American Barleywine,5


In [33]:
# take a look at the recipes_df
recipes_df.head()

,beerid,name,url,style,styleid,size_l,og,fg,abv,ibu,color,boilsize,boiltime,boilgravity,efficiency,mashthickness,sugarscale,brewmethod,pitchrate,primarytemp,primingmethod,primingamount,userid
0,1,Vanilla Cream Ale,/homebrew/recipe/view/1633/vanilla-cream-ale,Cream Ale,45,21.77,1.055,1.013,5.48,17.65,4.83,28.39,75,1.038,70.0,0.0,Specific Gravity,All Grain,0.0,17.78,corn sugar,4.5 oz,116.0
1,2,Southern Tier Pumking clone,/homebrew/recipe/view/16367/southern-tier-pumk...,Holiday/Winter Special Spiced Beer,85,20.82,1.083,1.021,8.16,60.65,15.64,24.61,60,1.07,70.0,0.0,Specific Gravity,All Grain,0.0,0.0,0,0,955.0
2,3,Zombie Dust Clone - EXTRACT,/homebrew/recipe/view/5920/zombie-dust-clone-e...,American IPA,7,18.93,1.063,1.018,5.91,59.25,8.98,22.71,60,0.0,70.0,0.0,Specific Gravity,extract,0.0,0.0,0,0,0.0
3,4,Zombie Dust Clone - ALL GRAIN,/homebrew/recipe/view/5916/zombie-dust-clone-a...,American IPA,7,22.71,1.061,1.017,5.8,54.48,8.5,26.5,60,0.0,70.0,0.0,Specific Gravity,All Grain,0.0,0.0,0,0,0.0
4,5,Bakke Brygg Belgisk Blonde 50 l,/homebrew/recipe/view/89534/bakke-brygg-belgis...,Belgian Blond Ale,20,50.0,1.06,1.01,6.48,17.84,4.57,60.0,90,1.05,72.0,0.0,Specific Gravity,All Grain,0.0,19.0,Sukkerlake,6-7 g sukker/l,18325.0


In [36]:
# How many rows are non-zero...
(recipes_df ==0).sum(axis=0)

beerid               0
name                 0
url                  0
style                0
styleid              0
size_l               0
og                   0
fg                   0
abv                 27
ibu               4159
color              101
boilsize             0
boiltime           283
boilgravity       2995
efficiency         101
mashthickness    29872
sugarscale           0
brewmethod           0
pitchrate        39303
primarytemp      22681
primingmethod        0
primingamount        0
userid           50490
dtype: int64

In [ ]:
# drop sparse columns
recipes_df = recipes_df.drop(columns = ["mashthickness", "pitchrate", "primarytemp", "userid"])

In [41]:
(recipes_df ==0).sum(axis=0)

beerid              0
name                0
url                 0
style               0
styleid             0
size_l              0
og                  0
fg                  0
abv                27
ibu              4159
color             101
boilsize            0
boiltime          283
boilgravity      2995
efficiency        101
sugarscale          0
brewmethod          0
primingmethod       0
primingamount       0
dtype: int64

In [42]:
# take a look at the reviews_df
reviews_df.sample(10)

,id,brewery_name,beer_style,brewery_id,review_overall,review_aroma,review_appearance,review_palate,review_taste,beer_abv,beer_beerid,beer_name,review_count
18992,19000,Vale Brewery Company,Extra Special / Strong Bitter (ESB),6985.0,3.5,3.5,3.5,3.5,3.5,5.0,25560,Hadda's Head Banger,2.0
31159,31181,Alandale Brewing Company,Oatmeal Stout,14268.0,4.125,3.75,4.125,4.0,4.125,4.5,34190,Oatmeal Stout Ale,4.0
9045,9041,Robert Cain & Company Limited,Oatmeal Stout,2930.0,4.0,4.0,4.5,4.5,4.0,4.1,40513,Cains Creamy Stout,1.0
34467,34498,Jupiter,American Porter,1497.0,3.625,3.875,3.875,3.625,4.0,7.0,15204,Porter,4.0
4561,4558,McKenzie Brew House,Belgian Pale Ale,1872.0,4.666666666666667,3.8333333333333335,3.5,4.0,4.0,5.1,5819,Belgian Pale Ale,3.0
8281,8271,Mountain Ridge Brewery,American Pale Ale (APA),23965.0,3.6666666666666665,3.3333333333333335,3.8333333333333335,3.5,3.8333333333333335,4.8,62505,Broughton Pale Ale,3.0
3256,3251,Mikkeller ApS,American Barleywine,13307.0,4.0,4.5,4.0,4.0,4.0,15.02,56372,BW5,1.0
44434,44487,Trapp Family Lodge Brewery,Vienna Lager,22512.0,4.25,3.9,4.05,4.15,4.15,5.0,57738,Trapp Vienna Lager,10.0
11619,11622,Craftsman Brewing Co.,Saison / Farmhouse Ale,2321.0,4.0,4.0,4.5,4.0,4.0,5.9,74434,Craftsman Harvest Saison,1.0
20123,20129,21st Amendment Brewery,American Double / Imperial Stout,735.0,1.25,3.5,3.75,3.0,1.5,7.5,60563,Hendrik's Russian Imperial Stout,2.0
